In [7]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
import urllib

# вытащить ДФы для одного, пары, тройки героев из сайта по дате
def get_df_avgElo_heroes(day_match, df):
       
    # создать ДФ для одного героя, пары и тройки
    df_one = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 1), :]  
    df_one = df_one.reset_index().drop('index', axis=1)
#     df_double = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 2), :]  
#     df_triple = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 3), :]  

    return df_one#, df_double, df_triple
# вытащить таблицу с elo героев для команды 
def get_df_AvgElo_heroes_teams(radiant_or_dire):
    # id команды из данных по матчу
    id_team = int(main[radiant_or_dire + '_team_id'][index])
    # дата матча
    date_match = date.fromtimestamp(main['start_time'][index])
    
    # создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
    url = ('http://www.datdota.com/api/heroes/elo?teams={}&tier=1&valve-event=does-not-matter&threshold=5'.format(id_team) +
        '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
        '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
        '&after=01%2F01%2F2011'+ 
        '&before={}%2F{}%2F{}'.format(date_match.day-1, date_match.month, date_match.year) + 
        '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    print(url)
    # выгрузить все с сайта и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_url = pd.DataFrame(dat)
    
    # создать ДФ для соло героев за все время игр команды (сигнатурки)
    return get_df_avgElo_heroes(date_match, df_url)

# Подгрузка мейн ДФ с id матчей

In [2]:
# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
main = main[main['match_id'] == 3364308762]
main

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
1506,3364308762,1502233763,2163.0,19,1838315.0,16,True,73.0,40.0,111.0,...,34505203.0,82262664.0,101356886.0,72312627.0,87278757.0,89117038.0,116585378.0,101450083.0,169025618.0,The International 2017


In [3]:
# создать основной ДФ, где будут записаны все матчи (id матча и rda героя)
df_AvgElo_heroes = pd.DataFrame()

# создать список названий колонок всех героев
all_her = main.loc[:,'radiant_H1':'dire_H5'].columns
for index in main.index:
    # ДФ для соединения всех герове матча в один ДФ 
    df_heroe_match = pd.DataFrame()
    # дата матча
    date_match = date.fromtimestamp(main['start_time'][index])
    # предыдущий день
    date_match =  date_match - timedelta(1)
    # создать дату два месяца назад от даты матча
    two_month_ago = date_match - timedelta(60)
    # создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
    url = ('http://www.datdota.com/api/heroes/elo?tier=1&valve-event=does-not-matter&threshold=10' +
        '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
        '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
        '&after={}%2F{}%2F{}'.format(two_month_ago.day, two_month_ago.month, two_month_ago.year) + 
        '&before={}%2F{}%2F{}'.format(date_match.day, date_match.month, date_match.year) + 
        '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    # выгрузить все с сайта и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_url = pd.DataFrame(dat)
    # создать ДФ для одного героя за 2 предыдущих месяца
    df_data_tabel_for_heroes = get_df_avgElo_heroes(date_match, df_url)
    
    for her in all_her:
        # вытащить id героя
        id_hero = main[her][index]

        # вытащить avg elo для данного героя
        avgElo_hero = df_data_tabel_for_heroes.loc[(index for index, x in enumerate(
                                            df_data_tabel_for_heroes['heroes']) if x == [id_hero]),'eloShift']
        
        # проверить есть ли герой
        try:
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0
        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer')
    df_AvgElo_heroes = pd.concat([df_AvgElo_heroes, df_heroe_match])
df_AvgElo_heroes = df_AvgElo_heroes.reset_index().drop('index', axis=1)
df_AvgElo_heroes['match_id'] = main['match_id']

In [4]:
df_AvgElo_heroes#.to_csv('../tabel/table from Datdota/tabel with AvgElo heroes (meta).csv')


,radiant_H1_AvgElo,radiant_H2_AvgElo,radiant_H3_AvgElo,radiant_H4_AvgElo,radiant_H5_AvgElo,dire_H1_AvgElo,dire_H2_AvgElo,dire_H3_AvgElo,dire_H4_AvgElo,dire_H5_AvgElo,match_id
0,-3.536543,-0.780075,1.564617,1.559056,3.584911,-0.607232,6.550237,8.844409,-4.591789,2.608368,NaN


In [5]:
url

'http://www.datdota.com/api/heroes/elo?tier=1&valve-event=does-not-matter&threshold=10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either&after=26%2F4%2F2017&before=25%2F6%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200'

# Вытащить как команды играют за все время (сигнатурки)

In [8]:
# создать основной ДФ, где будут записаны все матчи (id матча и rda героя)
df_AvgElo_heroes_teams = pd.DataFrame()

# создать список названий колонок  героев radiant
all_her_rad = main.loc[:,'radiant_H1':'radiant_H5'].columns
# создать список названий колонок  героев radiant
all_her_dir = main.loc[:,'dire_H1':'dire_H5'].columns

for index in main.index:
    # ДФ для соединения всех герове radiant & dire матча в один ДФ 
    df_heroe_match = pd.DataFrame()
    
    # ДЛЯ РЕДИАНТ
    # создать ДФ c avgElo для редиант по дате игры
    try: # если нету данных по команде то вернуть пустой ДФ
        df_data_tabel_radiant = get_df_AvgElo_heroes_teams('radiant')
    except:
        df_data_tabel_radiant = pd.DataFrame()
        
    for her in all_her_rad:
        # вытащить id героя
        id_hero = main[her][index]
        # проверить есть ли герой
        try:
             # вытащить avg elo для данного героя
            avgElo_hero = df_data_tabel_radiant.loc[(index for index, x in enumerate
                                                (df_data_tabel_radiant['heroes']) if x == [id_hero]),'eloShift']
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0
        
        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer') 
    
    # ДЛЯ ДАЕР
    # создать ДФ c avgElo для dire по дате игры
    try: # если нету данных по команде то вернуть пустой ДФ
        df_data_tabel_dire = get_df_AvgElo_heroes_teams('dire')
    except:
        df_data_tabel_dire = pd.DataFrame()
        
    for her in all_her_dir:
        # вытащить id героя
        id_hero = main[her][index]

        # проверить есть ли герой
        try:
            # вытащить avg elo для данного героя
            avgElo_hero = df_data_tabel_dire.loc[(index for index, x in enumerate
                                                (df_data_tabel_dire['heroes']) if x == [id_hero]),'eloShift']
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0

        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer') 
    df_AvgElo_heroes_teams = pd.concat([df_AvgElo_heroes_teams, df_heroe_match])
       
df_AvgElo_heroes_teams = df_AvgElo_heroes_teams.reset_index().drop('index', axis=1)
df_AvgElo_heroes_teams['match_id'] = main['match_id']

http://www.datdota.com/api/heroes/elo?teams=2163&tier=1&valve-event=does-not-matter&threshold=5&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either&after=01%2F01%2F2011&before=8%2F8%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200
http://www.datdota.com/api/heroes/elo?teams=1838315&tier=1&valve-event=does-not-matter&threshold=5&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either&after=01%2F01%2F2011&before=8%2F8%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200


In [6]:
df_AvgElo_heroes_teams#.to_csv('../tabel/table from Datdota/tabel with AvgElo heroes for teams (signatures).csv')

,radiant_H1_AvgElo,radiant_H2_AvgElo,radiant_H3_AvgElo,radiant_H4_AvgElo,radiant_H5_AvgElo,dire_H1_AvgElo,dire_H2_AvgElo,dire_H3_AvgElo,dire_H4_AvgElo,dire_H5_AvgElo,match_id
0,-1.383448,0,7.804802,0.965511,10.919285,10.646349,-4.112085,17.611371,0,9.22398,NaN
